In [1]:
%matplotlib inline 
import pandas as pd
import json,csv
import urllib2
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import geoplotlib as gp
import scipy as sp
import datetime
from geoplotlib.utils import BoundingBox
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.preprocessing import scale
from pylab import *
from geoplotlib.colors import colorbrewer
from geoplotlib.core import BatchPainter
from geoplotlib.layers import BaseLayer
from geoplotlib.colors import create_set_cmap

In [2]:
df_inc=pd.read_csv('SFPD_Incidents_-_from_1_January_2003.csv').dropna();
focus_areas=['PROSTITUTION','VEHICLE THEFT'];
df_focus=df_inc[df_inc['Category'].isin(focus_areas)];
df_03_focus=df_focus.loc[df_focus['Date'].str.contains("2003")];
df_15_focus=df_focus.loc[df_focus['Date'].str.contains("2015")];
ser_03_total=df_inc[df_inc['Date'].str.contains("2003")].groupby(['PdDistrict'], as_index=False).count();
ser_15_total=df_inc[df_inc['Date'].str.contains("2015")].groupby(['PdDistrict'], as_index=False).count();
#df_03_focus=df_03_focus[['Category','PdDistrict']];
#df_15_focus=df_15_focus[['Category','PdDistrict']];


In [11]:
df_03_focus=df_03_focus.groupby(['PdDistrict','Category'], as_index=False).count();
df_03_focus.rename(columns = {'X':'Counts'},inplace=True);
df_15_focus=df_15_focus.groupby(['PdDistrict','Category'], as_index=False).count();
df_15_focus.rename(columns = {'X':'Counts'},inplace=True);
df_03_focus=df_03_focus[['PdDistrict','Category','Counts']];
df_15_focus=df_15_focus[['PdDistrict','Category','Counts']];
ser_03_total.rename(columns = {'Category':'Counts'},inplace=True);
ser_15_total.rename(columns = {'Category':'Counts'},inplace=True);
ser_03_total=ser_03_total[['PdDistrict','Counts']];
ser_15_total=ser_15_total[['PdDistrict','Counts']];
dict_03=dict();
#print ser_03_total.set_index('PdDistrict').T.to_dict('records');
dict03=ser_03_total.set_index('PdDistrict').T.to_dict('records')[0];
dict15=ser_15_total.set_index('PdDistrict').T.to_dict('records')[0];


In [ ]:
#print df_03_focus.set_index('PdDistrict').T.to_dict()
with open('03total.csv', 'wb') as f:  # Just use 'w' mode in 3.x
    writer = csv.writer(f)
    for row in sorted(dict03.iteritems()):
        writer.writerow(row)
with open('15total.csv', 'wb') as f:  # Just use 'w' mode in 3.x   
    writer = csv.writer(f)
    for row in sorted(dict15.iteritems()):
        writer.writerow(row)

In [5]:
df_03_focus.shape

(17277, 13)

In [6]:
df_15_focus.shape

(8320, 13)

In [12]:
df_03_focus.head()

,PdDistrict,Category,Counts
0,BAYVIEW,PROSTITUTION,11
1,BAYVIEW,VEHICLE THEFT,2121
2,CENTRAL,PROSTITUTION,70
3,CENTRAL,VEHICLE THEFT,1193
4,INGLESIDE,PROSTITUTION,5


In [95]:
df_15_focus

,PdDistrict,Category,Counts
0,BAYVIEW,PROSTITUTION,3
1,BAYVIEW,VEHICLE THEFT,665
2,CENTRAL,PROSTITUTION,28
3,CENTRAL,VEHICLE THEFT,402
4,INGLESIDE,PROSTITUTION,3
5,INGLESIDE,VEHICLE THEFT,952
6,MISSION,PROSTITUTION,28
7,MISSION,VEHICLE THEFT,808
8,NORTHERN,PROSTITUTION,29
9,NORTHERN,VEHICLE THEFT,655
